In [2]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [28]:
puts "==status count=="
puts Task.group(:status).count
puts "==running task=="

puts Task.where(status:"run").map {|x| [x.id,x.name,x.runner,Time.now-x.updated_at].join(" | ")}.join("\n")


==status count==
{"open"=>20, "abort"=>3, "run"=>4, "close"=>58}
==running task==
2742 | data-import-block 15030000 - 15040000 | deepnote-runner | 186.65579108
2743 | data-import-block 15040000 - 15050000 | deepnote-runner | 92.885473912
2744 | data-import-block 15050000 - 15060000 | deepnote-runner | 88.350671471
2745 | data-import-block 15060000 - 15070000 | worker | 30.368151079


In [ ]:
def abort_reset_to_open()
  Task.where(status:"abort").map {|x| x.status="open"; x.save}
end

def task_clean()
  Task.destroy_all
end

def task_hello_task(count)
  count.times {|x|
    code = <<~TASKCODE

    puts "hello world!"
    _log "hello world!\n"
    
    TASKCODE
    Task.create_task("hello world task-#{x}", code)
  }
end


:task_hello_task

In [ ]:
abort_reset_to_open

[true]

In [ ]:
task_hello_task(10)

10

In [26]:
puts Task.where(status:"abort").last.output

2022-02-14 23:21:51 +0000 runner deepnote-runner take task 2738 : data-import-block 14990000 - 15000000
2022-02-14 23:21:51 +0000 == begin run ==
Exception Class: Errno::EPIPE
Exception Message: Failed to open TCP connection to nd-945-619-721.p2pify.com:443 (Broken pipe)
Exception Backtrace:
/usr/local/lib/ruby/2.7.0/resolv.rb:914:in `write'
/usr/local/lib/ruby/2.7.0/resolv.rb:914:in `print'
/usr/local/lib/ruby/2.7.0/resolv.rb:914:in `send'
/usr/local/lib/ruby/2.7.0/resolv.rb:665:in `request'
/usr/local/lib/ruby/2.7.0/resolv.rb:531:in `block in fetch_resource'
/usr/local/lib/ruby/2.7.0/resolv.rb:1124:in `block (3 levels) in resolv'
/usr/local/lib/ruby/2.7.0/resolv.rb:1122:in `each'
/usr/local/lib/ruby/2.7.0/resolv.rb:1122:in `block (2 levels) in resolv'
/usr/local/lib/ruby/2.7.0/resolv.rb:1121:in `each'
/usr/local/lib/ruby/2.7.0/resolv.rb:1121:in `block in resolv'
/usr/local/lib/ruby/2.7.0/resolv.rb:1119:in `each'
/usr/local/lib/ruby/2.7.0/resolv.rb:1119:in `resolv'
/usr/local/lib/ruby

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>